In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-25 22:40:20--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.54MB/s    in 0.6s    

2022-01-25 22:40:21 (1.54 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('encoding', 'UTF-8').csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(10)

In [ ]:
total_rows = df.count()
print(f"The total number of records is {total_rows}.")

## Examine the schema

In [ ]:
df.printSchema()

## Rename columns

In [ ]:
df_review_id = df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
df_review_id.show(5)

In [ ]:
df_review_id.printSchema()

In [13]:
from pyspark.sql.functions import to_date

In [14]:
review_df = df_review_id.withColumnRenamed('REVIEW_DATED', 'review_dated') 

In [18]:
products_df = df.select(['product_id', 'product_title'])

In [19]:
products_df = products_df.distinct()

In [20]:
total_rows = products_df.count()
print(f"The total nember of records after removing duplicates {total_rows}")

The total nember of records after removing duplicates 1780


In [ ]:
from pyspark.sql.functions import desc
customers_df = df.select(['customer_id'])\
  .groupby("customer_id")\
  .agg({"customer_id": "count"})\
  .orderBy(desc("count(customer_id)"))

customers_df.show()

In [ ]:
total_rows = customers_df.count()
print(total_rows)

In [ ]:
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'])
vine_df.show()

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/my_data_class_db"
config = {"user":"",
          "password": "",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table

review_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)